In [1]:
# General imports
import numpy as np
import torch
import matplotlib.pyplot as plt
# DeepMoD stuff
from deepymod import DeepMoD
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold, PDEFIND
from deepymod.training import train
from deepymod.training.sparsity_scheduler import TrainTestPeriodic
from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

# Settings for reproducibility
np.random.seed(44)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Making dataset

noise = 0.2
A = 1
v = 0.25

runs = 1
dataset = Dataset(BurgersDelta, A=A, v=v)

In [2]:
x = (np.random.rand(1,15)-0.5)*4
t = np.linspace(0.1, 1.1, 100) 
t_grid, x_grid = np.meshgrid(t, x, indexing='ij')
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=0, noise=noise, random=True, normalize=False)
X, y = X.to(device), y.to(device)
network = NN(2, [30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-7) # in terms of write iterations
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) # Defining optimizer

train(model, X, y, optimizer, sparsity_scheduler, log_dir='runs/figure_5', split=0.8, write_iterations=25, max_iterations=100000, delta=1e-5, patience=200) 
print(model.constraint_coeffs(sparse=True, scaled=True))

 86500  MSE: 2.36e-03  Reg: 6.18e-06  L1: 1.54e+00 

KeyboardInterrupt: 

In [3]:
#for n_x in np.array([4,6,8,10,12,14,16]):
for n_x in np.array([10,12,14,16,20]):
    for run in np.arange(runs):
        
       

 99975  MSE: 3.25e-05  Reg: 7.04e-06  L1: 1.35e+00 Algorithm converged. Writing model to disk.
[tensor([[ 0.0000],
        [ 0.0000],
        [ 0.7474],
        [ 0.0000],
        [ 0.0000],
        [-0.5980],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]], device='cuda:0', grad_fn=<MulBackward0>)]
 70525  MSE: 2.35e-04  Reg: 1.48e-05  L1: 2.71e+00 Algorithm converged. Writing model to disk.
[tensor([[ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.6818],
        [-0.4993],
        [ 0.0000],
        [ 0.0000],
        [-1.5243],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]], device='cuda:0', grad_fn=<MulBackward0>)]
 81250  MSE: 1.87e-05  Reg: 1.16e-06  L1: 1.63e+00 Algorithm converged. Writing model to disk.
[tensor([[ 0.0000],
        [ 0.0000],
        [ 0.8893],
        [ 0.0000],
        [ 0.0000],
        [-0.7388],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],

In [31]:
#number_of_samples = 1000

#idx = np.random.permutation(y.shape[0])
#X = torch.tensor(X[idx, :][:number_of_samples], dtype=torch.float32, requires_grad=True)
#y = torch.tensor(y[idx, :][:number_of_samples], dtype=torch.float32)

In [38]:
X, y = X.to(device), y.to(device)

In [ ]:
for run in np.arange(runs):
    network = NN(2, [30, 30, 30, 30], 1)
    library = Library1D(poly_order=2, diff_order=3) # Library function
    estimator = Threshold(0.2) # Sparse estimator 
    constraint = LeastSquares() # How to constrain
    model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

    sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=500, delta=1e-7) # in terms of write iterations
    optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) # Defining optimizer

    train(model, X, y, optimizer, sparsity_scheduler, log_dir='runs/x_rand_b_1000', split=0.8, write_iterations=25, max_iterations=100000, delta=1e-4, patience=500) 
    print(model.constraint_coeffs(sparse=True, scaled=True))

 18775  MSE: 5.72e-05  Reg: 9.18e-06  L1: 2.44e+00 